# Порядковые статистики

Часто, для массива данных необходимо посчитать порядковые статистики. Если исходный массив содержит n элементов, то k-тая порядоковая статистика - число, которое стоит на k-том месте в упорядоченном изначальном массиве. 

Например: минимум - это 1-ая порядковая статистика, максимум - n-ая порядковая статистика, а медиана - n/2 - порядковая статистика.

In [1]:
#include <iostream>
#include <vector>
#include <chrono>
#include <cstdlib>
#include <algorithm>

In [2]:
typedef std::chrono::milliseconds Milliseconds;
typedef std::chrono::steady_clock Clock;
typedef Clock::time_point Time;

In [3]:
srand(time(NULL)); // для генерации чисел

(void) @0x7ffc6f385410


In [4]:
void gen(std::vector<int> & array, int n) {
    array.clear();
    for(int i = 0; i < n; i++) {
        array.push_back(rand()%n);
    }
} 

In [5]:
void print(std::vector<int> & array){
    for(auto i : array){
        std::cout << i << ' ';
    }
    std::cout << std::endl;
}
// вспомогательные функции, чтобы удобнее работать с массивом

In [6]:
void print_stat(std::vector<int> array, int k) {
    std::sort(array.begin(), array.end());
    for(int i = 0; i < array.size(); i++) {
        if(i == k) {
            std::cout << '|' << array[i] << '|' << ' ';
        } else {
            std::cout << array[i] << ' ';   
        }
    }
}

# Подход "В лоб"

Как видно из определения порядковой статистики, для ее нахождения можно просто упорядочить весь массив и после этого выбрать нужный элемент в массиве.

In [7]:
int trivial_statistic(std::vector<int> data, int k){
    std::sort(data.begin(), data.end()); // Сортируем изначальный массив
    return data[k]; // Берем k-тый элемент
}

In [8]:
std::vector<int> array;
gen(array, 10);
print(array);
print_stat(array, 4);

1 8 5 9 2 1 8 8 3 1 
1 1 1 2 |3| 5 8 8 8 9 

(void) @0x7ffc6f385410


In [9]:
std::cout << trivial_statistic(array, 4);

3

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


In [10]:
gen(array, 100000);
Time start = Clock::now();
trivial_statistic(array, 30);
unsigned long t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << "В массиве из 100000 статистика нашлась за " << t << " миллисекунд" << std::endl;

В массиве из 100000 статистика нашлась за 557 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


Данный метод крайне прост и понятен, однако, из предыдущей лекции мы знаем, что любая сортировка, основанная на сравнениях, не может работать быстрее, чем за $N*log(N)$. Таким образом и наш алгоритм будет работать не быстрее чем $N*log(N)$.

# Минимальный и максимальный элемент

Однако мы точно знаем, что некоторые порядковые статистики мы может находить гораздо быстрее - например, минимум и максимум:
* Создадим изначально новую переменную где будет в итоге лежать минимальный элемент и положим туда изначально первый элемент массива
* Далее будем идти по массиву, и если встретим число меньшее, то запишем в нашу переменную это новое число
* Таким образом, в конце в этой переменной будет храниться число, которое меньше всех в этом массиве, а это по определению минимум

Аналогично для максимума.

Видно, что данный алгоритм будет работать всего за $N$ операций, нежели $N*log(N)$ при сортировке.

In [11]:
int min_elem(std::vector<int> data) {
    int result = data[0];
    for(int num : data){
        if(num < result) {
            result = num;
        }
    }
    return result;
}

In [12]:
gen(array, 11);
print_stat(array, 0);

|0| 0 1 3 3 4 5 6 6 9 10 

(void) @0x7ffc6f385410


In [13]:
std::cout << min_elem(array) << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


In [18]:
gen(array, 100000);
start = Clock::now();
min_elem(array);
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << "В массиве из 100000 статистика нашлась за " << t << " миллисекунд" << std::endl;

В массиве из 100000 статистика нашлась за 35 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


Возникает правомерный вопрос: если мы можем посчитать крайние статистики за N, то как дела обстоят с другими порядковыми статистиками? Можем ли мы находить их также эффективно.

Хорошая новость заключается в том, что да, действительно можем!

# Выбор разбиением

Вспомним подход, реализуемый в алгоритме быстрой сортировки - каждый раз мы делили массив на две части таким образом, чтобы все элементы справа были больше, а все элементы слева меньше разделительного числа и дальше применяли эту же операцию к получившимся подмассивам.

Идея поиска статистики примерно такая же:
* Пускай мы поделили массив на две части. В таком случае если в левом подмассиве элементов больше чем k, то это означает, что k-ая порядковая статистика лежим в нем. Если же меньше, то это означаем, что k-ая статистика лежив в правом подмассиве
* Если же в левом оказалось ровно k-1 элемент, то значит наш опорный элемент и есть k-тая порядковая статистика

In [14]:
template<class Iter>
int partition_statistic(Iter begin, Iter end, int k) {
    int distance = end - begin;
    if(distance <= 1) return *(begin); // если остался один элемент, значит, что он и есть нужная статистика

    int t = * (begin + distance / 2 );
    Iter middle = std::partition(begin, end, [t](int s){ return s < t;});
    std::partition(middle, end, [t](int s){ return s == t; });
    // функцию partition мы уже реализовывали в предыдущей лекции, поэтому не будем углубляться в ее устройство и просто
    // воспользуемся стандартной функцией из STL
    // Также нам нужно не просто разбить на 2 массива, а также еще, чтобы граничный элемент был на границе
    // Поэтому проведем еще одну операцию partition, которая перетащит все граничные элементы на границу двух подмассивов

    int left_size = middle - begin;
    if(left_size == k) { // нашли нашу статистику
        return t;
    } else if(left_size > k) {
        return partition_statistic(begin, middle, k);
    } else {
        return partition_statistic(middle+1, end, k-left_size-1);
        // мы отняли от k размер левоц части, так как относительно правого подмассива наша статистика теперь стоит
        // на left_size-1 позиций раньше
        // мы не сделали так для левого массива, так как его начало совпадает с началом исходного массива и наша
        // статистика все еще находится на правильной позиции
    }
}

int partition_statistic(std::vector<int> data, int k) {
    return partition_statistic(data.begin(), data.end(), k);
    // перегрузим функцию, так как если сразу работать с итераторами, то после нахождения статистики, исходный массив
    // будет изменен, а в этом случае он будет скопирован
}

In [15]:
gen(array, 11);
print_stat(array, 3);

0 0 0 |2| 2 3 3 3 10 10 10 

(void) @0x7ffc6f385410


In [16]:
std::cout << partition_statistic(array, 3);

2

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


In [19]:
gen(array, 100000);
start = Clock::now();
partition_statistic(array, 30);
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << "В массиве из 100000 статистика нашлась за " << t << " миллисекунд" << std::endl;

В массиве из 100000 статистика нашлась за 99 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


Видно, что данный алгоритм значительно быстрее наивной реализации через сортировку.

Если бы мы каждый раз делили массив на две равные части, то тогда бы нам было бы необходимо сделать 
$N+N/2+N/4+N/8+... = N*(1+1/2+1/4+1/8+...) < N*2$. Таким образом, алгоритм работает в среднем за $N$ действий.

Однако, как и быстрая сортировка, данная реализация имеет недостаток: при неудачных данных она может работать за $N^2$ действий.

Полученный результат уже очен  неплох, но можем ли мы найти нужную порядковую статистику гарантированно за N действий, а не только при удачном расположении данных?

Ответ - да!

# Медиана медиан

Данный алгоритм уже достаточно сложен для понимания, поэтому приведем только его общую схему.

По сути этот алгоритм - это просто улучшенная версия предыдущего. Основным отличием от него является то, как мы выбираем опорный элемент, по которому мы делим наше множество. В предыдущем алгоритме мы просто выбирали серединный (с таким же успехом, мы могли выбирать случайный элемент), и поэтому при плохих данных алгоритм может проседать по скорости. В данном же алгоритме мы выбираем опорный элемент более интеллектуально.

* Поделим весь массив на небольшие массивы размера 5. (Если его размер не делится на 5, то оставшиеся несколько элементов, мы потом просто добавим к следующей стадии)
* Отсортируем каждый их этих подмассивов любым алгоритмом сортировки и возьмем срединный элемент в каждом подмассиве (по сути мы нашли медиану в каждом из подмассивов)
 * Важным момент тут заключается в том, что хоть мы и используем здесь сортировку, но это не влияет на зависимость скорости работы всего алгоритма от количества данных, так как мы все время сортируем всего 5 элементов. Пример, если бы мы использовали даже алгоритм сортировки пузырьком: $ 5^2 + 5^2 + 5^2 + ... = N/5 * 5^2 = N * 5 $
* Далее составим массив из этих медиан и припишем к ней оставшиеся элементы после первого пункта (если таковые имеются)
* Рекурсивно посчитаем нашу функцию от получившегося массива - то есть найдем медиану найденных медиан.
* Полученная медиана М - и есть тот самый "хороший" элемент, по которому необходимо делить наше множество.
* Далее все также как и в предыдущем алгоритме - делим множества по этому элементу, смотрим, на размер получившихся множеств и далее определяем, где находится наша статистика и либо вызываем нашу функцию от нужного множества или сразу возвращаем медиану.

In [20]:
template <class Iter>
int median_of_medians(Iter begin, Iter end, int k) {
    if(end - begin <= 10) { // если осталось мало элементов, то просто отсортируем и вернем k-ый элемент
        std::sort(begin, end);
        return *(begin + k);
    }

    std::vector<int> medians; // массив из медиан
    Iter prev = begin;
    Iter next = prev + 5; // делим по 5 элементов
    while(next < end) {
        medians.push_back(median_of_medians(prev, next, 2)); // добавляем медиану 5-ти элементного множества
        prev += 5;
        next += 5;
    }

    if(next != end) { // если остались еще элементы, то дописываем их
        std::copy(prev, end, std::back_inserter(medians));
    }

    int M = median_of_medians(medians.begin(), medians.end(), int(medians.size())/2); // находим медиану медиан

    // далее идет точь-в-точь предыдущий алгоритм
    Iter middle = std::partition(begin, end, [M](int s){return s < M;});
    std::partition(middle, end, [M](int s) {return s == M;});

    int left_size = middle - begin;
    if(left_size == k) { // нашли нашу статистику
        return M;
    } else if(left_size > k) {
        return median_of_medians(begin, middle, k);
    } else {
        return median_of_medians(middle+1, end, k-left_size-1);
    }
}

int median_of_medians(std::vector<int> data, int k) {
    return median_of_medians(data.begin(), data.end(), k);
}

In [25]:
gen(array, 22);
print_stat(array, 7);

0 0 4 5 5 5 6 |6| 7 7 8 8 9 9 10 11 13 14 17 17 20 21 

(void) @0x7ffc6f385410


In [26]:
std::cout << median_of_medians(array, 7) << std::endl;

6


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


In [27]:
gen(array, 100000);
start = Clock::now();
median_of_medians(array, 30);
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << "В массиве из 100000 статистика нашлась за " << t << " миллисекунд" << std::endl;

В массиве из 100000 статистика нашлась за 327 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


Математика говорит, что подобный выбор опорного элемента гарантирует отброс минимум четверти всех элементов (в отличии от предыдущего алгоритма, который в некоторых случаях мог отбросить только 1 элемент). Таким образом он всегда работает за линейное время N относительно размера данных.

Однако, как можно было заметить, хоть предварительный поиск медианы медиан и не влияет на линейность работы, однако все равно повышает время работы в какое-то константное количество раз. 

Это можно видеть на результатах работы - на небольшом массиве обычный алгоритм справился за 99 миллисекунд, а медиана медиан только за 327 миллисекунд. 

Таким образом, целесообразно использовать медиану медиан на больших массивах, где есть большая вероятность плохой конфигурации данных.

In [28]:
gen(array, 10000000);
std::sort(array.begin(), array.end());
std::reverse(array.begin(), array.end());
start = Clock::now();
partition_statistic(array, 0); // попробуем найти минимум в этом массиве
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << "В массиве из 10000000 статистика нашлась за " << t << " миллисекунд" << std::endl;

В массиве из 10000000 статистика нашлась за 9402 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


In [30]:
gen(array, 1000000);
std::sort(array.begin(), array.end());
std::reverse(array.begin(), array.end());
start = Clock::now();
median_of_medians(array, 0); // попробуем найти минимум в этом массиве
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << "В массиве из 10000000 статистика нашлась за " << t << " миллисекунд" << std::endl;

В массиве из 10000000 статистика нашлась за 3004 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa449ff96e0


Как можно заметить, если запускать оба алгоритма на больших данных, которые обратно отсортированы и искать при этом минимальный элемент, то тогда обычный алгоритм работает за 9402 секунды против 3004 секунд у медианы медиан.